## Data Validation

#### Purpose: Ensures that the data is correct, accurate, and meets business or technical rules.

In [26]:
# Import the BigQuery client library from Google Cloud
from google.cloud import bigquery

In [27]:
# Initialize BigQuery client
client = bigquery.Client()

Check for duplicate entries in customers table (key: customer_id)

In [28]:
query = """
SELECT COUNT(*) AS null_count
FROM `concise-faculty-452613-a1.brazil_e_commerce.customers`
WHERE customer_id IS NULL;
"""

# Execute the query
query_job = client.query(query)  # Send SQL query to BigQuery
results = query_job.result()  # Fetch query results

# Print the results
for row in results:
    print(f"Null count: {row.null_count}")

Null count: 0


Check for data ranges of review score in order_review table

In [29]:
query = """
SELECT COUNT(*) AS out_of_range_count
FROM `concise-faculty-452613-a1.brazil_e_commerce.order_reviews`
WHERE review_score < 5 OR review_score > 0;
"""
query_job = client.query(query)  # Send SQL query to BigQuery
results = query_job.result()  # Fetch query results

# Print the results
for row in results:
    print(row)

Row((99224,), {'out_of_range_count': 0})


Check for duplicate entries in order_items table (composite key: order_id, order_item_id)

In [17]:
query = """
SELECT COUNT(*) AS total_duplicate_composite_keys
FROM (
    SELECT order_id, order_item_id
    FROM `concise-faculty-452613-a1.brazil_e_commerce.order_items`
    GROUP BY order_id, order_item_id
    HAVING COUNT(*) > 1
) AS duplicates;
"""

query_job = client.query(query)  # Send SQL query to BigQuery
results = query_job.result()  # Fetch query results

# Print the result
for row in results:
    print(f"Total number of duplicate composite keys: {row['total_duplicate_composite_keys']}")

Total number of duplicate composite keys: 0


Check for duplicate entries in product table ( key: product_id)

In [36]:
query = """
SELECT product_id, COUNT(*) AS count
FROM `concise-faculty-452613-a1.brazil_e_commerce.products`
GROUP BY product_id
HAVING COUNT(*) > 1;
"""

query_job = client.query(query)  # Send SQL query to BigQuery
results = query_job.result()  # Fetch query results

if results.total_rows == 0:
    print("No duplicates found.")
else:
    # Print the result if duplicates exist
    for row in results:
        print(f"Product ID: {row['product_id']} has {row['count']} duplicates")

No duplicates found.


Check for duplicate entries in order_payments table (composite key: order_id, payment_sequential)

In [43]:
query = """
SELECT order_id, payment_sequential, COUNT(*) AS count
FROM `concise-faculty-452613-a1.brazil_e_commerce.order_payments`
GROUP BY order_id, payment_sequential
HAVING COUNT(*) > 1;
"""
query_job = client.query(query)  # Send SQL query to BigQuery
results = query_job.result()  # Fetch query results

if results.total_rows == 0:
    print("No duplicates found.")
else:
    # Print the result if duplicates exist
    for row in results:
        print(f"Product ID: {row['product_id']} has {row['count']} duplicates")

No duplicates found.


Check for invalid order dates where order_approved_at is greater than order_delivered_customer_date in orders table

In [38]:
query = """
SELECT COUNT(*) AS invalid_dates
FROM `concise-faculty-452613-a1.brazil_e_commerce.orders`
WHERE order_approved_at > order_delivered_customer_date
AND order_approved_at IS NOT NULL
AND order_delivered_customer_date IS NOT NULL;
"""

query_job = client.query(query)  # Send SQL query to BigQuery
results = query_job.result()  # Fetch query results

# Check if there are results
if results.total_rows == 0:
    print("No invalid dates found.")
else:
    # Print the result if invalid dates exist
    for row in results:
        print(f"Total number of invalid dates: {row['invalid_dates']}")

Total number of invalid dates: 61


#### There are 61 records where the order_approved_at date is greater than the order_delivered_customer_date in orders table.

In [39]:
query = """
SELECT order_id, order_approved_at, order_delivered_customer_date
FROM `concise-faculty-452613-a1.brazil_e_commerce.orders`
WHERE order_approved_at > order_delivered_customer_date;
"""
query_job = client.query(query)  # Send SQL query to BigQuery
results = query_job.result()  # Fetch query results
for row in results:
    print(row)

Row(('6b80bb20190715d71c43efff617bd659', datetime.datetime(2017, 3, 1, 10, 51, 46), datetime.datetime(2017, 2, 24, 14, 27, 26)), {'order_id': 0, 'order_approved_at': 1, 'order_delivered_customer_date': 2})
Row(('f6f0b2497c5a4ca89670186757ab2684', datetime.datetime(2017, 7, 5, 17, 9, 11), datetime.datetime(2017, 7, 3, 17, 53, 29)), {'order_id': 0, 'order_approved_at': 1, 'order_delivered_customer_date': 2})
Row(('1fab4ac9d85079b3da72a11475ae1685', datetime.datetime(2017, 9, 13, 22, 6, 11), datetime.datetime(2017, 9, 8, 20, 13, 3)), {'order_id': 0, 'order_approved_at': 1, 'order_delivered_customer_date': 2})
Row(('c3b8c17ee15e0e798c2e178b7d4c7f42', datetime.datetime(2017, 9, 13, 22, 8, 55), datetime.datetime(2017, 9, 11, 16, 43, 46)), {'order_id': 0, 'order_approved_at': 1, 'order_delivered_customer_date': 2})
Row(('bc4854efd86d9f42140c951c595d20c1', datetime.datetime(2017, 9, 13, 22, 0, 51), datetime.datetime(2017, 9, 6, 22, 15, 45)), {'order_id': 0, 'order_approved_at': 1, 'order_deliv

#### This is an invalid data scenario, as the order approval date should logically occur before the order delivery date.

#### Due to discrepancies identified during data validation, the column order_approved_at has been excluded from the fact table to ensure data integrity and consistency in analysis.

Check for invalid order dates where order_purchase_timestamp is greater than order_delivered_customer_date

In [46]:
query = """
SELECT COUNT(*) AS invalid_dates
FROM `concise-faculty-452613-a1.brazil_e_commerce.orders`
WHERE order_purchase_timestamp > order_delivered_customer_date;
"""

query_job = client.query(query)  # Send SQL query to BigQuery
results = query_job.result()  # Fetch query results

# Check if there are results
if results.total_rows == 0:
    print("No invalid dates found.")
else:
    # Print the result if invalid dates exist
    for row in results:
        print(f"Total number of invalid dates: {row['invalid_dates']}")

Total number of invalid dates: 0
